In [1]:
import requests
import pandas as pd
import time
from datetime import datetime
import os

In [ ]:
# Function to apply limits to temperature values
def apply_limits(temp_value, lower_limit, upper_limit):
    try:
        if temp_value < lower_limit:
            return "Opps low"
        elif temp_value > upper_limit:
            return "Opps high"
        return temp_value
    except Exception as e:
        print(f"Error applying limits: {e}")
        return temp_value  # Return original value in case of error

# Define limits
lower_limit = 15  # Lower temperature limit
upper_limit = 25  # Upper temperature limit

# Check if the source file exists before entering the loop
if not os.path.exists('blynk_data.csv'):
    raise FileNotFoundError("The file blynk_data.csv does not exist!")

# Loop to simulate handling dynamically updated data
while True:
    try:
        # Load dynamically changing CSV data into pandas DataFrame
        df = pd.read_csv('blynk_data.csv')
        print(df)

        # Check if necessary columns are present in the data
        if 'Temperature' not in df.columns or 'Timestamp' not in df.columns:
            print(f"Available columns: {df.columns}")
            raise ValueError("Missing required columns: 'Temperature' or 'Timestamp'")

        # Apply limiting function to temperature column
        df['Temperature'] = df['Temperature'].apply(lambda x: apply_limits(x, lower_limit, upper_limit))

        # Write the updated DataFrame back to the same CSV file
        csv_file_path = "E:/Aditi/Programming/demo.csv"

        try:
            df.to_csv(csv_file_path, index=False)  # Save DataFrame to CSV
        except PermissionError:
            print(f"Permission denied: Unable to write to {csv_file_path}. Please close the file if it is open.")
            continue  # Skip to the next iteration if the file is open or locked
        except Exception as e:
            print(f"Error writing to CSV file: {e}")
            continue  # Skip to the next iteration on error

        # Reload the updated data to check for alerts
        try:
            df_reloaded = pd.read_csv(csv_file_path)
        except FileNotFoundError:
            print(f"File {csv_file_path} not found. Skipping this iteration.")
            continue
        except Exception as e:
            print(f"Error reading the CSV file: {e}")
            continue

        # Check for any "Opps high" alerts in the data
        alert_rows = df_reloaded[df_reloaded['Temperature'] == 'Opps high']
        if not alert_rows.empty:
            for index, row in alert_rows.iterrows():
                # Print the specific timestamp of the alert row
                print(f"Alert system provoked at {row['Timestamp']}")
        else:
            print("No alerts generated.")

    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except ValueError as e:
        print(f"Data format error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    # Sleep or wait before checking again (adjust time as per your needs)
    time.sleep(60)  # Wait for 60 seconds before the next iteration


   Temperature  Humidity                   Timestamp
0         18.1      68.0  2024-09-16 18:56:42.845981
No alerts generated.
